### End to End Demo for Cleanlab Studio Python API

1) Initialize Studio object with your API key

In [1]:
from cleanlab_studio import Studio

api_key = "3a746f025ab14260a18ffe3188c1eece"
studio = Studio(api_key)

2. Load dataset as pandas dataframe

In [14]:
import pandas as pd

df = pd.read_csv("https://cleanlab-public.s3.amazonaws.com/StudioDemoDatasets/goemotions_10.csv")
print(df.head())

                                                text       id         author   
0                                 Man I love reddit.  eeibobj  MrsRobertshaw  \
1  Right? Considering it’s such an important docu...  eespn2i   ImperialBoss   
2  That's crazy; I went to a super [RELIGION] hig...  ed5tx8y     Beachy5313   
3                                that's adorable asf  ef961hv   RedRails1917   
4  "Sponge Blurb Pubs Quaw Haha GURR ha AAa!" fin...  edl7cr3       Tiagulus   

              subreddit    link_id   parent_id   created_utc  rater_id   
0              facepalm  t3_ahulml   t3_ahulml  1.547965e+09        18  \
1            TrueReddit  t3_aizyuz  t1_eesoak0  1.548280e+09        61   
2  TeenMomOGandTeenMom2  t3_ac10u9  t1_ed5tfyc  1.546536e+09        23   
3  traaaaaaannnnnnnnnns  t3_akz2ia   t3_akz2ia  1.548764e+09        73   
4          youtubehaiku  t3_acuvb0  t1_edc21ip  1.546984e+09        54   

   example_very_unclear      label  
0                 False       love  


3. Upload dataset to Studio

In [15]:
dataset_id = studio.upload_dataset(df, "Go Emotions")
dataset_id

Uploading dataset...: 100%|██████████|
Generating schema...: 100%|██████████|
Ingesting Dataset...: 100%|██████████|


'8f2cac759aef4f7d88f9f67826dede3b'

4. Create Project

In [4]:
project_id = studio.create_project(dataset_id, "Multilabel project", "text", model_type="fast", task_type="multi-label")
project_id

'27e1f5faade94c17af8889f4cd902a50'

5. Get cleanset ID for project

In [5]:
cleanset_id = studio.get_latest_cleanset_id(project_id)
cleanset_id

'ec23e413f0f74afb838aba2c15aac70d'

6. Poll for cleanset status until cleanset is ready

In [6]:
studio.poll_cleanset_status(cleanset_id, timeout=30)

Cleanset Progress: | Step 0/5, Initializing...


KeyboardInterrupt: 

In [7]:
cleanlab_columns_df = studio.download_cleanlab_columns(cleanset_id)
cleanlab_columns_df.head()

APIError: Cannot download cleanlab columns. Cleanset is not ready. You can check cleanset status at https://app.cleanlab.ai/ or by using the Python API `poll_cleanset_status()` method

7. Download cleanlab columns

In [8]:
cleanlab_columns_df = studio.download_cleanlab_columns("c3e1f577fe804cea96cee770f074a27b")
cleanlab_columns_df.head()

,index,cleanlab_issue,cleanlab_label_quality,cleanlab_suggested_label,cleanlab_clean_label
0,9316,True,0.050548,⭐⭐⭐⭐,None
1,9852,True,0.050820,⭐⭐⭐⭐⭐,None
2,14706,True,0.050994,⭐⭐⭐⭐⭐,None
3,11168,True,0.051186,⭐⭐⭐⭐⭐,None
4,217,True,0.175522,⭐⭐⭐⭐,None


In [9]:
corrected_df = studio.apply_corrections("c3e1f577fe804cea96cee770f074a27b", df)

In [11]:
df.shape

(17500, 4)

In [12]:
corrected_df[corrected_df.apply(lambda x: x["stars"] != df.iloc[x["index"]]["stars"], axis=1)]

,index,review_text,asin,stars
240,240,This is the best woman's magazine out there: a...,B00005N7QC,⭐⭐⭐⭐⭐
250,250,Your system would not let me cancel this order...,B00005N7QC,⭐
320,320,I ordered this magazine and did not receive on...,B00005N7RD,⭐
322,322,I have loved BH & G for 40 yrs it seems!,B00005N7OV,⭐⭐⭐⭐⭐
519,519,"Always loved this mag, great to be able to tot...",B00005N7OV,⭐⭐⭐⭐⭐
...,...,...,...,...
16896,16896,One of my favorite magazines I've purchased fo...,B00VATZJPQ,⭐⭐⭐⭐⭐
17012,17012,Excellent mag. Lots of great articles. My favo...,B00YQH98G0,⭐⭐⭐⭐⭐
17047,17047,I've been subscribing to this magazine since i...,B00ZSGXN3W,⭐⭐⭐⭐⭐
17247,17247,Always surprised at the current and useful inf...,B016N11NR8,⭐⭐⭐⭐


In [13]:
df[df.apply(lambda x: x["index"] not in corrected_df["index"], axis=1)]

,index,review_text,asin,stars
9316,9316,good thank you,B000HWY1PG,⭐⭐⭐
9852,9852,I have always loved Family Circle and to have ...,B000IJ84F6,⭐⭐⭐⭐
11168,11168,love this month's recipes!,B000IOEK7M,⭐⭐⭐⭐
14706,14706,I love the magazine and gain inspiration for m...,B00005QJE0,⭐
